In [ ]:
%matplotlib widget
import geopandas
import pandas
import starepandas
import pystare
import shapely
import matplotlib.pyplot as plt
import plots
import numpy

In [ ]:
starepandas.__version__

In [ ]:
cities = {'name': ['midway', 'Fiji', 'Baker', 'honolulu'],
          'lat': [28.2, -17.8,  0.2, 21.3282956], 
          'lon': [-177.35, 178.1, -176.7, -157.9]}
                   
cities = starepandas.STAREDataFrame(cities)
sids = starepandas.sids_from_xy(cities.lon, cities.lat, resolution=1)
cities.set_sids(sids, inplace=True)
trixels = cities.make_trixels(wrap_lon=False)
cities.set_trixels(trixels, inplace=True)

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.grid(True)

antimeridian = shapely.geometry.LineString([(180,-90), (180,90)])

ax.plot(*antimeridian.xy, color='red')
ax.set_ylim(-90,90)
ax.set_xlim(90,270)

cities.plot(ax=ax, column='name')

## Multipolygon

In [ ]:
combined = starepandas.STAREDataFrame(sids=[sids, sids])
trixels = combined.make_trixels(wrap_lon=False)
combined.set_trixels(trixels, inplace=True)

# Algorithm

In [ ]:
bbox = shapely.geometry.Polygon([(-180, -90), (180, -90), (180, 90), (-180, 90)])

In [ ]:
trixels = geopandas.GeoSeries(combined[combined._trixel_column_name])

In [ ]:
def split(trixels):
    bbox = shapely.geometry.Polygon([(-180, -90), (180, -90), (180, 90), (-180, 90)])

    trixels = geopandas.GeoSeries(trixels)

    inside = trixels.explode(index_parts=True).reset_index(drop=True)
    inside = inside.intersection(bbox)
    inside = geopandas.tools.collect(inside)

    outside = trixels.explode(index_parts=True).reset_index(drop=True)
    outside = outside.difference(bbox)
    outside = outside.apply(lambda x: shapely.affinity.translate(x, xoff=-360))
    outside = geopandas.tools.collect(outside)

    split = inside.union(outside)
    return split

# Usage

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.grid(True)

ax.set_ylim(-90,90)
ax.set_xlim(-190,190)

cities.split_antimeridian(inplace=True)

cities.plot(ax=ax, column='name')

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.grid(True)

ax.set_ylim(-90, 90)
ax.set_xlim(-190, 190)

combined.split_antimeridian(inplace=True)
#combined.trixels.explode(index_parts=True).reset_index(drop=True)


combined.plot(ax=ax)

In [ ]:
cities.trixels[0].geoms[1].exterior.xy

In [ ]:
cities = {'name': ['a', 'b'],
          'lat': [-50, 50], 
          'lon': [60, -360]}

cities = starepandas.STAREDataFrame(cities)
sids = starepandas.sids_from_xy(cities.lon, cities.lat, resolution=1)
cities.set_sids(sids, inplace=True)
#trixels = cities.make_trixels(wrap_lon=True, wrap_thresh=180)
trixels = cities.make_trixels(wrap_lon=False)
cities.set_trixels(trixels, inplace=True)
#cities.split_antimeridian(inplace=True)

In [ ]:
fig, ax = plt.subplots(dpi=100)
ax.grid(True)

antimeridian = shapely.geometry.LineString([(180,-90), (180, 90)])

#ax.plot(*antimeridian.xy, color='red')
#ax.set_ylim(-90,90)
#ax.set_xlim(90,270)

cities.plot(ax=ax, column='name')

In [ ]:
cities
for i, v in cities.iterrows():    
    g = v.trixels
    if not g.exterior.is_ccw:
        print(i)
        x = (numpy.array(g.exterior.xy[0]) + 180) % 360.0 - 180
        y = g.exterior.xy[1]
        cities.trixels[i] = shapely.geometry.Polygon(zip(x,y))

In [ ]:
for i, geom in cities.set_geometry('trixels').geometry.iteritems():
    print(i)

In [ ]:
vs = starepandas.to_vertices(sids, wrap_lon=False, wrap_thresh=270)
v1 = numpy.array([vs[0][1]-vs[0][0], vs[1][1]-vs[1][0]])
v2 = numpy.array([vs[0][2]-vs[0][0], vs[1][2]-vs[1][0]])
numpy.cross(v1,v2) > 0

In [ ]:
lats = vs[0]
lons = vs[1]
lons

In [ ]:
vs

In [ ]:
corners = starepandas.vertices2corners(vs)
corners

In [ ]:
v1 = numpy.array([corners[:,1,0] - corners[:,0,0], corners[:,1,1] - corners[:,0,1]])
v2 = numpy.array([corners[:,2,0] - corners[:,0,0], corners[:,2,1] - corners[:,0,1]])
numpy.cross(v1,v2) >1

In [ ]:
corners